In [1]:
#data management tools
import pandas as pd
import numpy as np

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import ADASYN

import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sb
import re
import datetime

# #Vectorization and Tokenizing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn import utils

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer


# #Models
# from sklearn.ensemble import AdaBoostClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
# from sklearn.naive_bayes import GaussianNB
# from sklearn.linear_model import LinearRegression

# #Metrics and Testing
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

import gensim
from gensim.models import word2vec
from gensim.models import Doc2Vec
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

import multiprocessing
cores = multiprocessing.cpu_count()


In [2]:
df1 = pd.read_csv('/Users/ken.clements/code/praxis/CERT42/logon.csv')
df2 = pd.read_csv('/Users/ken.clements/code/praxis/CERT42/device.csv')
df3 = pd.read_csv('/Users/ken.clements/code/praxis/CERT42/email.csv')
df4 = pd.read_csv('/Users/ken.clements/code/praxis/CERT42/file.csv')
df5 = pd.read_csv('/Users/ken.clements/code/praxis/CERT42/http.csv')
ans = pd.read_csv('/Users/ken.clements/code/praxis/CERT42/answers/answers.csv')

In [3]:
START = datetime.date(2010,7,1)
END = datetime.date(2010,8, 30)

In [4]:
df1['date'] = pd.to_datetime(df1['date'])
df2['date'] = pd.to_datetime(df2['date'])
df3['date'] = pd.to_datetime(df3['date'])
df4['date'] = pd.to_datetime(df4['date'])
df5['date'] = pd.to_datetime(df5['date'])

df1['day_date'] = pd.to_datetime(df1['date']).dt.date
df2['day_date'] = pd.to_datetime(df2['date']).dt.date
df3['day_date'] = pd.to_datetime(df3['date']).dt.date
df4['day_date'] = pd.to_datetime(df4['date']).dt.date
df5['day_date'] = pd.to_datetime(df5['date']).dt.date

In [5]:
df3['activity'] = "email"
df4['activity'] = "file"
df5['activity'] = "http"

df1_bs = df1[((df1['day_date'] >= START) & (df1['day_date'] <= END))]
df2_bs = df2[((df2['day_date'] >= START) & (df2['day_date'] <= END))]
df3_bs = df3[((df3['day_date'] >= START) & (df3['day_date'] <= END))]
df4_bs = df4[((df4['day_date'] >= START) & (df4['day_date'] <= END))]
df5_bs = df5[((df5['day_date'] >= START) & (df5['day_date'] <= END))]

In [6]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
def prepare_text(pretext):

    # tokenize mission.
    tokens = word_tokenize(pretext, language="english", preserve_line=True)

    # Parts of speech (POS) tag tokens.
    token_tag = pos_tag(tokens)
    
    # Only include some of the POS tags.
    include_tags = ['VBN', 'VBD', 'JJ', 'JJS', 'JJR', 'CD', 'NN', 'NNS', 'NNP', 'NNPS']
    filtered_tokens = (tok for tok, tag in token_tag if tag in include_tags)    
    
    #stem words.
    #stemmed_tokens = (lancaster.stem(tok) for tok in filtered_tokens)

    #lemm words.
    #lemmed_words = (lemmatizer.lemmatize(tok) for tok in stemmed_tokens)

    #lower_words = [str.lower(tok) for tok in lemmed_words]
    lower_words = [str.lower(tok) for tok in filtered_tokens]

    #x = cv.fit_transform(tokens).toarray()

    #joined = ", ".join(lower_words)

    return(lower_words)

#ps = PorterStemmer()
#lancaster = LancasterStemmer()
#lemmatizer = nltk.stem.WordNetLemmatizer()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ken.clements/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ken.clements/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/ken.clements/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [7]:
df3_bs['df3tokenized'] = df3_bs['content'].apply(prepare_text)
# df4_bs['df4tokenized'] = df4_bs['content'].apply(prepare_text)
# df5_bs['df5tokenized'] = df5_bs['content'].apply(prepare_text)

/var/folders/p4/svf85_b50jvfz4dvm3yf5h0c0000gq/T/ipykernel_8124/2382823816.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3_bs['df3tokenized'] = df3_bs['content'].apply(prepare_text)


In [8]:
df_merged = pd.concat([df1_bs, df2_bs, df3_bs])
# df_merged = pd.concat([df1_bs, df2_bs, df3_bs, df4_bs, df5_bs])

In [9]:
df_merged['label'] = 'normal'

In [10]:
df_merged.loc[df_merged['id'].isin(ans['id']),['label']] = 'bad'

In [40]:
df = df_merged.dropna(subset=['df3tokenized'])


In [41]:
df = df[['df3tokenized','user']]

In [44]:
train, test = train_test_split(df, test_size=0.3, random_state=42)

In [45]:
train_tagged = train.apply(lambda r: TaggedDocument(words=r['df3tokenized'], tags=r['user']), axis=1)
test_tagged = test.apply(lambda r: TaggedDocument(words=r['df3tokenized'], tags=r['user']), axis=1)

In [46]:
model_dbow = Doc2Vec(dm=0, vector_size=5, negative=5, hs=0, min_count=2, sample = 0, workers=cores)


In [47]:
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

100%|██████████| 228943/228943 [00:00<00:00, 4045144.14it/s]


In [48]:
for epoch in range(5):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 228943/228943 [00:00<00:00, 4963848.75it/s]


In [32]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags, model.infer_vector(doc.words)) for doc in sents])
    return targets, regressors

In [33]:
y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)

In [ ]:
X_train

In [34]:
logreg = LogisticRegression(n_jobs=1, C=1e5, max_iter=100)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)


In [35]:
from sklearn.metrics import accuracy_score, f1_score
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

Testing accuracy 0.9994904147005167
Testing F1 score: 0.9992356869866146


In [39]:
cf_matrix = metrics.confusion_matrix(y_test, y_pred)
print(cf_matrix)

[[    0    50]
 [    0 98069]]


In [51]:
blsmote = BorderlineSMOTE()
X_train_blsmote, y_train_blsmote = blsmote.fit_resample(X_train, y_train)

In [54]:
logreg.fit(X_train_blsmote, y_train_blsmote)
y_pred = logreg.predict(X_test)

In [55]:
y_pred = logreg.predict(X_test)


In [56]:
from sklearn.metrics import accuracy_score, f1_score
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

Testing accuracy 0.981226877567036
Testing F1 score: 0.9900374613589615


In [57]:
cf_matrix = metrics.confusion_matrix(y_test, y_pred)
print(cf_matrix)

[[   41     9]
 [ 1833 96236]]


In [59]:
adasyn = ADASYN()
X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train, y_train)

In [60]:
logreg.fit(X_train_adasyn, y_train_adasyn)
y_pred = logreg.predict(X_test)

In [61]:
y_pred = logreg.predict(X_test)


In [1]:
from sklearn.metrics import accuracy_score, f1_score
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing recall %s' % recall_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

NameError: name 'y_test' is not defined

In [63]:
cf_matrix = metrics.confusion_matrix(y_test, y_pred)
print(cf_matrix)

[[   42     8]
 [ 1980 96089]]


In [ ]:
dates = df_merged['day_date']
dates = dates.unique()

In [ ]:
dfwow = []
for d in dates:
    dayevents = df_merged.loc[(df_merged['day_date'] == d)]
    users = dayevents['user']
    users = users.unique()
    for u in users:
        userd = dayevents[(dayevents['user'] == u)]
        date = d
        username = u
        login_count = userd[userd['activity'] == 'Logon'].shape[0]
        logout_count = userd[userd['activity'] == 'Logoff'].shape[0]
        email_count = userd[userd['to'].notnull()].shape[0]
        file_count = userd[userd['filename'].notnull()].shape[0]
        http_count = userd[userd['url'].notnull()].shape[0]
        start_time = userd['date'].iloc[0]
        end_time = userd['date'].iloc[-1]

        if (userd[(userd['label'] == 'bad')].shape[0] > 0): label = 1
        else: label = 0
        dfwow.append([date, username, login_count, logout_count, email_count, file_count, http_count, label])


In [ ]:
final = pd.DataFrame(dfwow, columns=['date', 'username', 'login_count', 'logout_count', 'email_count', 'file_count', 'http_count', 'tfidf3_mean', 'tfidf4_mean', 'tfidf5_mean', 'label'])
final = final.dropna(subset=['tfidf5_mean', 'tfidf3_mean', 'tfidf4_mean'])
final['date'] = (final['date'] - final['date'].min())  / np.timedelta64(1,'D')
label_encoder = LabelEncoder()
final['username'] = label_encoder.fit_transform(final['username'])


In [ ]:
X = final.loc[:, final.columns != 'label']
y = final['label']

In [ ]:
final_adasyn = ADASYN()
X_final_adasyn, y_final_adasyn = final_adasyn.fit_resample(X, y)
final_adasyn = X_final_adasyn.copy()
final_adasyn['label'] = y_final_adasyn

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5,stratify=y)

classifier = LogisticRegression(max_iter=3)

classifier.fit(X_train,y_train)

predicted = classifier.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
#print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))